In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 14,
   "id": "initial_id",
   "metadata": {
    "collapsed": true,
    "ExecuteTime": {
     "end_time": "2024-08-12T14:42:42.988905400Z",
     "start_time": "2024-08-12T14:42:42.978914800Z"
    }
   },
   "outputs": [],
   "source": [
    "##\n",
    "## NUEVA REFACTORIZACION\n",
    "##\n",
    "def load_data():\n",
    "\n",
    "    import pandas as pd\n",
    "\n",
    "    dataframe = pd.read_csv(\n",
    "        \"sentences.csv.zip\",\n",
    "        index_col=False,\n",
    "        compression=\"zip\",\n",
    "    )\n",
    "\n",
    "    data = dataframe.phrase\n",
    "    target = dataframe.target\n",
    "\n",
    "    return data, target"
   ]
  },
  {
   "cell_type": "code",
   "outputs": [],
   "source": [
    "def make_train_test_split(x, y):\n",
    "\n",
    "    from sklearn.model_selection import train_test_split\n",
    "\n",
    "    (x_train, x_test, y_train, y_test) = train_test_split(\n",
    "        x,\n",
    "        y,\n",
    "        test_size=0.25,\n",
    "        random_state=123456,\n",
    "    )\n",
    "    return x_train, x_test, y_train, y_test"
   ],
   "metadata": {
    "collapsed": false,
    "ExecuteTime": {
     "end_time": "2024-08-12T14:42:43.061079900Z",
     "start_time": "2024-08-12T14:42:42.994084900Z"
    }
   },
   "id": "6c825f7cac878ea9",
   "execution_count": 15
  },
  {
   "cell_type": "code",
   "outputs": [],
   "source": [
    "def make_pipeline(estimator):\n",
    "\n",
    "    from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer\n",
    "    from sklearn.pipeline import Pipeline\n",
    "\n",
    "    vectorizer = CountVectorizer(\n",
    "        lowercase=True,\n",
    "        analyzer=\"word\",\n",
    "        token_pattern=r\"\\b[a-zA-Z]\\w+\\b\",\n",
    "        stop_words=\"english\",\n",
    "    )\n",
    "\n",
    "    transformer = TfidfTransformer()\n",
    "\n",
    "    pipeline = Pipeline(\n",
    "        steps=[\n",
    "            (\"vectorizer\", vectorizer),\n",
    "            (\"transformer\", transformer),\n",
    "            (\"estimator\", estimator),\n",
    "        ],\n",
    "        verbose=False,\n",
    "    )\n",
    "\n",
    "    return pipeline"
   ],
   "metadata": {
    "collapsed": false,
    "ExecuteTime": {
     "end_time": "2024-08-12T14:42:43.095167600Z",
     "start_time": "2024-08-12T14:42:43.068182600Z"
    }
   },
   "id": "7c2ef03dc0ec4455",
   "execution_count": 16
  },
  {
   "cell_type": "code",
   "outputs": [],
   "source": [
    "def make_grid_search(estimator, param_grid, cv=5):\n",
    "\n",
    "    from sklearn.model_selection import GridSearchCV\n",
    "\n",
    "    grid_search = GridSearchCV(\n",
    "        estimator=estimator,\n",
    "        param_grid=param_grid,\n",
    "        cv=cv,\n",
    "        scoring=\"balanced_accuracy\",\n",
    "    )\n",
    "\n",
    "    return grid_search"
   ],
   "metadata": {
    "collapsed": false,
    "ExecuteTime": {
     "end_time": "2024-08-12T14:42:43.128768300Z",
     "start_time": "2024-08-12T14:42:43.101288100Z"
    }
   },
   "id": "3bae827805c4fc78",
   "execution_count": 17
  },
  {
   "cell_type": "code",
   "outputs": [],
   "source": [
    "def save_estimator(estimator):\n",
    "\n",
    "    import pickle\n",
    "\n",
    "    with open(\"estimator.pickle\", \"wb\") as file:\n",
    "        pickle.dump(estimator, file)"
   ],
   "metadata": {
    "collapsed": false,
    "ExecuteTime": {
     "end_time": "2024-08-12T14:42:43.130768800Z",
     "start_time": "2024-08-12T14:42:43.116782200Z"
    }
   },
   "id": "dc545692972c9e0a",
   "execution_count": 18
  },
  {
   "cell_type": "code",
   "outputs": [],
   "source": [
    "def load_estimator():\n",
    "\n",
    "    import os\n",
    "    import pickle\n",
    "\n",
    "    if not os.path.exists(\"estimator.pickle\"):\n",
    "        return None\n",
    "    with open(\"estimator.pickle\", \"rb\") as file:\n",
    "        estimator = pickle.load(file)\n",
    "\n",
    "    return estimator"
   ],
   "metadata": {
    "collapsed": false,
    "ExecuteTime": {
     "end_time": "2024-08-12T14:42:43.172480500Z",
     "start_time": "2024-08-12T14:42:43.134770Z"
    }
   },
   "id": "9c4c64292d26d048",
   "execution_count": 19
  },
  {
   "cell_type": "code",
   "outputs": [],
   "source": [
    "def train_logistic_regression():\n",
    "\n",
    "    from sklearn.linear_model import LogisticRegression\n",
    "    from sklearn.metrics import balanced_accuracy_score\n",
    "\n",
    "    data, target = load_data()\n",
    "\n",
    "    x_train, x_test, y_train, y_test = make_train_test_split(\n",
    "        x=data,\n",
    "        y=target,\n",
    "    )\n",
    "\n",
    "    pipeline = make_pipeline(\n",
    "        estimator=LogisticRegression(max_iter=1000),\n",
    "    )\n",
    "\n",
    "    estimator = make_grid_search(\n",
    "        estimator=pipeline,\n",
    "        param_grid={\n",
    "            \"transformer__norm\": [\"l1\", \"l2\", None],\n",
    "            \"transformer__use_idf\": [True, False],\n",
    "            \"transformer__smooth_idf\": [True, False],\n",
    "        },\n",
    "        cv=5,\n",
    "    )\n",
    "\n",
    "    estimator.fit(x_train, y_train)\n",
    "\n",
    "    best_estimator = load_estimator()\n",
    "\n",
    "    if best_estimator is not None:\n",
    "\n",
    "        saved_balanced_accuracy = balanced_accuracy_score(\n",
    "            y_true=y_test, y_pred=best_estimator.predict(x_test)\n",
    "        )\n",
    "\n",
    "        current_balanced_accuracy = balanced_accuracy_score(\n",
    "            y_true=y_test, y_pred=estimator.predict(x_test)\n",
    "        )\n",
    "\n",
    "        if current_balanced_accuracy < saved_balanced_accuracy:\n",
    "            estimator = best_estimator\n",
    "\n",
    "    save_estimator(estimator)\n",
    "\n",
    "\n",
    "train_logistic_regression()"
   ],
   "metadata": {
    "collapsed": false,
    "ExecuteTime": {
     "end_time": "2024-08-12T14:42:51.354172600Z",
     "start_time": "2024-08-12T14:42:43.144477Z"
    }
   },
   "id": "a77d3521aee71dbb",
   "execution_count": 20
  },
  {
   "cell_type": "code",
   "outputs": [
    {
     "data": {
      "text/plain": "array(['neutral', 'positive', 'positive', ..., 'negative', 'negative',\n       'negative'], dtype=object)"
     },
     "execution_count": 21,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "def use_estimator():\n",
    "\n",
    "    import pickle\n",
    "\n",
    "    import pandas as pd\n",
    "\n",
    "    dataframe = pd.read_csv(\n",
    "        \"sentences.csv.zip\",\n",
    "        index_col=False,\n",
    "        compression=\"zip\",\n",
    "    )\n",
    "\n",
    "    data = dataframe.phrase\n",
    "\n",
    "    with open(\"estimator.pickle\", \"rb\") as file:\n",
    "        estimator = pickle.load(file)\n",
    "\n",
    "    prediction = estimator.predict(data)\n",
    "\n",
    "    return prediction\n",
    "\n",
    "\n",
    "use_estimator()"
   ],
   "metadata": {
    "collapsed": false,
    "ExecuteTime": {
     "end_time": "2024-08-12T14:42:51.439684800Z",
     "start_time": "2024-08-12T14:42:51.360172300Z"
    }
   },
   "id": "c59eb484a87fe2d8",
   "execution_count": 21
  },
  {
   "cell_type": "code",
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Pipeline(steps=[('vectorizer',\n",
      "                 CountVectorizer(stop_words='english',\n",
      "                                 token_pattern='\\\\b[a-zA-Z]\\\\w+\\\\b')),\n",
      "                ('transformer', TfidfTransformer(norm=None, smooth_idf=False)),\n",
      "                ('estimator', LogisticRegression(max_iter=1000))]):\n",
      "  Balanced Accuracy: 0.7165 (0.9962)\n",
      "           Accuracy: 0.8233 (0.9982)\n"
     ]
    }
   ],
   "source": [
    "def check_estimator():\n",
    "\n",
    "    import pickle\n",
    "\n",
    "    import pandas as pd\n",
    "    from sklearn.metrics import accuracy_score, balanced_accuracy_score\n",
    "\n",
    "    data, target = load_data()\n",
    "\n",
    "    x_train, x_test, y_train_true, y_test_true = make_train_test_split(\n",
    "        x=data,\n",
    "        y=target,\n",
    "    )\n",
    "\n",
    "    with open(\"estimator.pickle\", \"rb\") as file:\n",
    "        estimator = pickle.load(file)\n",
    "\n",
    "    y_train_pred = estimator.predict(x_train)\n",
    "    y_test_pred = estimator.predict(x_test)\n",
    "\n",
    "    accuracy_train = round(accuracy_score(y_train_true, y_train_pred), 4)\n",
    "    accuracy_test = round(accuracy_score(y_test_true, y_test_pred), 4)\n",
    "    balanced_accuracy_train = round(\n",
    "        balanced_accuracy_score(y_train_true, y_train_pred), 4\n",
    "    )\n",
    "    balanced_accuracy_test = round(balanced_accuracy_score(y_test_true, y_test_pred), 4)\n",
    "\n",
    "    print(estimator.best_estimator_, \":\", sep=\"\")\n",
    "    print(f\"  Balanced Accuracy: {balanced_accuracy_test} ({balanced_accuracy_train})\")\n",
    "    print(f\"           Accuracy: {accuracy_test} ({accuracy_train})\")\n",
    "\n",
    "\n",
    "check_estimator()"
   ],
   "metadata": {
    "collapsed": false,
    "ExecuteTime": {
     "end_time": "2024-08-12T14:42:51.564221Z",
     "start_time": "2024-08-12T14:42:51.432687500Z"
    }
   },
   "id": "b5728b26cd65f669",
   "execution_count": 22
  },
  {
   "cell_type": "code",
   "outputs": [],
   "source": [
    "def train_mlp_classifier():\n",
    "\n",
    "    from sklearn.metrics import balanced_accuracy_score\n",
    "    from sklearn.neural_network import MLPClassifier\n",
    "\n",
    "    data, target = load_data()\n",
    "\n",
    "    x_train, x_test, y_train, y_test = make_train_test_split(\n",
    "        x=data,\n",
    "        y=target,\n",
    "    )\n",
    "\n",
    "    pipeline = make_pipeline(\n",
    "        estimator=MLPClassifier(max_iter=10000),\n",
    "    )\n",
    "\n",
    "    estimator = make_grid_search(\n",
    "        estimator=pipeline,\n",
    "        param_grid={\n",
    "            \"transformer__norm\": [\"l1\", \"l2\", None],\n",
    "            \"transformer__use_idf\": [True, False],\n",
    "            \"transformer__smooth_idf\": [True, False],\n",
    "            \"estimator__hidden_layer_sizes\": [(1,), (5,), (5, 5)],\n",
    "            \"estimator__solver\": [\"adam\"],\n",
    "            \"estimator__learning_rate_init\": [0.01, 0.001, 0.0001],\n",
    "        },\n",
    "        cv=5,\n",
    "    )\n",
    "\n",
    "    estimator.fit(x_train, y_train)\n",
    "\n",
    "    best_estimator = load_estimator()\n",
    "\n",
    "    if best_estimator is not None:\n",
    "\n",
    "        saved_balanced_accuracy = balanced_accuracy_score(\n",
    "            y_true=y_test, y_pred=best_estimator.predict(x_test)\n",
    "        )\n",
    "\n",
    "        current_balanced_accuracy = balanced_accuracy_score(\n",
    "            y_true=y_test, y_pred=estimator.predict(x_test)\n",
    "        )\n",
    "\n",
    "        if current_balanced_accuracy < saved_balanced_accuracy:\n",
    "            estimator = best_estimator\n",
    "\n",
    "    save_estimator(estimator)\n",
    "\n",
    "\n",
    "train_mlp_classifier()"
   ],
   "metadata": {
    "collapsed": false,
    "ExecuteTime": {
     "end_time": "2024-08-12T15:39:23.419246300Z",
     "start_time": "2024-08-12T14:42:51.560224Z"
    }
   },
   "id": "222c3586f410f07e",
   "execution_count": 23
  },
  {
   "cell_type": "code",
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Pipeline(steps=[('vectorizer',\n",
      "                 CountVectorizer(stop_words='english',\n",
      "                                 token_pattern='\\\\b[a-zA-Z]\\\\w+\\\\b')),\n",
      "                ('transformer',\n",
      "                 TfidfTransformer(norm=None, smooth_idf=False, use_idf=False)),\n",
      "                ('estimator',\n",
      "                 MLPClassifier(hidden_layer_sizes=(5,), max_iter=10000))]):\n",
      "  Balanced Accuracy: 0.7223 (0.9948)\n",
      "           Accuracy: 0.8163 (0.9976)\n"
     ]
    }
   ],
   "source": [
    "check_estimator()"
   ],
   "metadata": {
    "collapsed": false,
    "ExecuteTime": {
     "end_time": "2024-08-12T15:57:34.087605600Z",
     "start_time": "2024-08-12T15:57:33.989626800Z"
    }
   },
   "id": "2fbaba8eaa9ca0d3",
   "execution_count": 24
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 2
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython2",
   "version": "2.7.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}